In [1]:
from symred.symplectic_form import PauliwordOp
from symred.S3_projection import S3_projection, gf2_gaus_elim, gf2_basis_for_gf2_rref, QubitTapering

In [2]:
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf

# comment out due to incompatible versions of Cirq and OpenFermion in Orquestra
def QubitOperator_to_dict(op, num_qubits):
    assert(type(op) == of.QubitOperator)
    op_dict = {}
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:    
        letters = ['I' for i in range(num_qubits)]
        for i in t:
            letters[i[0]] = i[1]
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

# Two examples are provided -- comment out as necessary

# One for which Hartree-Fock identifies the correct symmetry sector...
geometry=[
    ("O",(0,0,0)),
    ("H",(0.952519,0,0)),
    ("H",(-0.246530058,0.9200627021,0))
]

# ...and one for which it does not.
#geometry=[
#    ('H', (0.0,0.0,0.0)),
#    ('H', (2.45366053071732,0.0,0.0)),
#    #('H', (2.45366053071732,2.45366053071732,0.0)),
#    #('H', (0.0,2.45366053071732,0.0))
#     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
#molecule.load()

# Run pyscf.
molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=1,run_ccsd=1,run_fci=1)

n_qubits    = 2*molecule.n_orbitals
n_electrons = molecule.n_electrons

ham_fermionic = of.get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_jw = of.jordan_wigner(ham_fermionic)

ham_dict = QubitOperator_to_dict(ham_jw, n_qubits)

print('Jordan-Wigner Hamiltonian:\n\n', ham_dict)

Jordan-Wigner Hamiltonian:

 {'IIIIIIIIIIIIII': (-46.40178201838023+0j), 'ZIIIIIIIIIIIII': (12.413911714677482+0j), 'YZYIIIIIIIIIII': (0.1250642657528696+0j), 'XZXIIIIIIIIIII': (0.1250642657528696+0j), 'YZZZZZYIIIIIII': (-0.042709517791887974+0j), 'XZZZZZXIIIIIII': (-0.042709517791887974+0j), 'YZZZZZZZZZYIII': (0.07314533074509919+0j), 'XZZZZZZZZZXIII': (0.07314533074509919+0j), 'IZIIIIIIIIIIII': (12.413911714677482+0j), 'IYZYIIIIIIIIII': (0.1250642657528696+0j), 'IXZXIIIIIIIIII': (0.1250642657528696+0j), 'IYZZZZZYIIIIII': (-0.042709517791887974+0j), 'IXZZZZZXIIIIII': (-0.042709517791887974+0j), 'IYZZZZZZZZZYII': (0.0731453307450992+0j), 'IXZZZZZZZZZXII': (0.0731453307450992+0j), 'IIZIIIIIIIIIII': (1.657782221605793+0j), 'IIYZZZYIIIIIII': (0.11811415718604787+0j), 'IIXZZZXIIIIIII': (0.11811415718604787+0j), 'IIYZZZZZZZYIII': (-0.28534861549869284+0j), 'IIXZZZZZZZXIII': (-0.28534861549869284+0j), 'IIIZIIIIIIIIII': (1.6577822216057931+0j), 'IIIYZZZYIIIIII': (0.11811415718604788+0j), 'III

In [3]:
import scipy

def get_ground_state(sparse_operator, initial_guess=None):
    """Compute lowest eigenvalue and eigenstate.
    Args:
        sparse_operator (LinearOperator): Operator to find the ground state of.
        initial_guess (ndarray): Initial guess for ground state.  A good
            guess dramatically reduces the cost required to converge.
    Returns
    -------
        eigenvalue:
            The lowest eigenvalue, a float.
        eigenstate:
            The lowest eigenstate in scipy.sparse csc format.
    """
    values, vectors = scipy.sparse.linalg.eigsh(sparse_operator,
                                                k=1,
                                                v0=initial_guess,
                                                which='SA',
                                                maxiter=1e7)

    order = np.argsort(values)
    values = values[order]
    vectors = vectors[:, order]
    eigenvalue = values[0]
    eigenstate = vectors[:, 0]
    return eigenvalue, eigenstate.T

In [4]:
H2O_op = PauliwordOp(ham_dict)
print(get_ground_state(H2O_op.to_sparse_matrix)[0])
hf_state = np.array([1,1,1,1,1,1,1,1,1,1,0,0,0,0])
qtap = QubitTapering(H2O_op)
taper_H2O_op = qtap.taper_it(ref_state=hf_state)
print(get_ground_state(taper_H2O_op.to_sparse_matrix)[0])

-75.01084664817124
-75.01084664817049


In [5]:
qtap.taper_reference_state(hf_state)

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

array([1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [74]:
hf_state = np.array([1,1,1,1,1,1,1,1,1,1,0,0,0,0])

def identify_symmetry_sector(symmetry_ops,ref_state):
    """ Given the specified reference state, determine the
    correspinding sector by measuring the symmetry generators
    """
    return (-1)**np.count_nonzero(symmetry_ops.Z_block & ref_state, axis=1)

sector = identify_symmetry_sector(stabs, hf_state)
project_op = proj_op.perform_projection(H2O_op, sector)
ham_sparse = project_op.to_sparse_matrix
ground_energy, ground_state = get_ground_state(ham_sparse)#, initial_guess=initial_guess)
print(ground_energy)

-75.0108466481707


In [36]:
import scipy
from itertools import product

for sector in [(-1,-1,1,1)]:# product([+1,-1], repeat=4):
    print(sector)
    project_op = proj_op.perform_projection(H2O_op, sector)
    ham_sparse = project_op.to_sparse_matrix
    ground_energy, ground_state = get_ground_state(ham_sparse)#, initial_guess=initial_guess)
    print(ground_energy)
ham_sparse = project_op.to_sparse_matrix

ground_energy, ground_state = get_ground_state(ham_sparse)#, initial_guess=initial_guess)
print(ground_energy)

-74.50545932232426


In [26]:
Jx,Jy,Jz=1,1,1
h=1
heisenberg = {'XXI':Jx, 'IXX':Jx, 'XIX':Jx, 
              'ZZI':Jz, 'IZZ':Jz, 'ZIZ':Jz,
              'YYI':Jy, 'IYY':Jy, 'YIY':Jy, 
              'ZII':h, 'IZI':h, 'IIZ':h}
heisenberg_op = PauliwordOp(heisenberg)
stabs = symmetry_generators(heisenberg_op)

In [27]:
print(stabs)

(1+0j) ZZZ


In [76]:
def J_term(sqp, N):
    q_pos = list(range(N))
    make_sqp = list(zip(q_pos[:-1], q_pos[1:]))
    out = []
    for i,j in make_sqp:
        blank = list('I'*N)
        blank[i],blank[j]=sqp,sqp
        out.append(''.join(blank))
    return out

def h_terms(N):
    make_Z = list(range(N))
    out = []
    for i in make_Z:
        blank = list('I'*N)
        blank[i]='Z'
        out.append(''.join(blank))
    return out

N=23
terms=[]
for sqp in ['X', 'Y', 'Z']:
    terms+=J_term(sqp, N)
#terms+=h_terms(N)

heisenberg = {op:1 for op in terms}
heisenberg_op = PauliwordOp(heisenberg)
stabs = symmetry_generators(heisenberg_op)
print(stabs)

(1+0j) XXXXXXXXXXXXXXXXXXXXXXX +
(1+0j) ZZZZZZZZZZZZZZZZZZZZZZZ


In [72]:
proj_op = S3_projection(stabs, 'Z')
project_op = proj_op.perform_projection(heisenberg_op, [-1])

In [73]:
project_op.PauliwordOp_to_OF()

[(1-0j) [Z1],
 (2+0j) [Z0 Z1],
 (1+0j) [X0],
 (-1+0j) [X0 Z1],
 (2+0j) [X0 X1],
 (2+0j) [Y0 Y1]]